In [1]:
# Required Libraries

import google.generativeai as genai
from openai import OpenAI
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain_community.llms import OpenAI
# from langchain_google_genai import ChatGoogleGenerativeAI

C:\Users\itsme\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Required Secret Key
from config import gemini_key, open_ai_key

In [3]:
def input_image_setup(file_loc):
    from pathlib import Path

    if not (img := Path(file_loc)).exists():
        raise FileNotFoundError(f"Could not find image: {img}")

    image_parts = [
        {
            "mime_type": "image/jpeg",
            "data": Path(file_loc).read_bytes()
            }
        ]
    return image_parts
def get_image_info(image_loc, prompt):
    genai.configure(api_key=gemini_key)
    # Set up the model
    generation_config = {
        "temperature":0,
        "top_p":1,
        "top_k":32,
        "max_output_tokens":4096,
    }
    
    model = genai.GenerativeModel(model_name="gemini-pro-vision", generation_config=generation_config)

    input_prompt = """ You are an expert in data visualization and graph analysis, adept at interpreting graphical data and generating structured JSON configurations for Plotly"""

    question_prompt = prompt

    image_prompt = input_image_setup(image_loc)
    prompt_parts = [input_prompt, image_prompt[0], question_prompt]
    response = model.generate_content(prompt_parts)
    return str(response.text)

In [10]:
import plotly.graph_objs as go
import plotly.io as pio
import json
import re

def extract_json_section(input_string, tag):
    """Extract JSON section between specified XML-like tags."""
    pattern = f"<{tag}>(.*?)</{tag}>"
    match = re.search(pattern, input_string, re.DOTALL)
    if match:
        return match.group(1).strip()
    return None

def parse_json(json_str):
    """Attempt to parse JSON with relaxed rules."""
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        # Try replacing single quotes with double quotes and parsing again
        try:
            fixed_json_str = json_str.replace("'", '"')
            return json.loads(fixed_json_str)
        except json.JSONDecodeError as e:
            raise ValueError(f"Invalid JSON: {e}")

def plot_from_ai_output_v3(input_string):
    # Extract JSON sections
    data_json = extract_json_section(input_string, "data")
    layout_json = extract_json_section(input_string, "layout")
    config_json = extract_json_section(input_string, "config")

    # Parse JSON strings with relaxed rules
    data = parse_json(data_json) if data_json else None
    layout = parse_json(layout_json) if layout_json else None
    config = parse_json(config_json) if config_json else None

    if not data or not layout:
        raise ValueError("Invalid or missing data or layout JSON.")

    # Prepare traces for the plot
    traces = []
    for trace_data in data:
        trace_type = trace_data.get('type')

        if trace_type == 'bar':
            trace = go.Bar(
                x=trace_data.get('x', []),
                y=trace_data.get('y', []),
                name=trace_data.get('name', ''),
                marker=dict(color=trace_data['marker']['color']) if 'marker' in trace_data else None,
                text=trace_data.get('text', ''),
                hoverinfo=trace_data.get('hoverinfo', 'x+y+name'),
                orientation=trace_data.get('orientation', 'v'),
                offsetgroup=trace_data.get('offsetgroup', None),
                base=trace_data.get('base', None)
            )
            if trace_data.get('stackgroup'):  # For stacked bar charts
                trace.update(barmode='stack')

        elif trace_type == 'line':
            trace = go.Scatter(
                x=trace_data.get('x', []),
                y=trace_data.get('y', []),
                mode='lines',
                name=trace_data.get('name', ''),
                line=dict(color=trace_data['line']['color'], width=trace_data['line']['width']) if 'line' in trace_data else None,
                hoverinfo='text',
                text=trace_data['text'] if 'text' in trace_data else None
            )

        elif trace_type == 'scatter':
            trace = go.Scatter(
                x=trace_data.get('x', []),
                y=trace_data.get('y', []),
                mode=trace_data.get('mode', 'markers'),
                name=trace_data.get('name', ''),
                marker=dict(color=trace_data['marker']['color']) if 'marker' in trace_data else None,
                line=dict(color=trace_data['line']['color'], width=trace_data['line']['width']) if 'line' in trace_data else None,
                hoverinfo='text',
                text=trace_data['text'] if 'text' in trace_data else None
            )

        elif trace_type == 'pie':
            trace = go.Pie(
                labels=trace_data.get('labels', []),
                values=trace_data.get('values', []),
                name=trace_data.get('name', ''),
                textinfo=trace_data.get('textinfo', 'percent+label'),
                hoverinfo=trace_data.get('hoverinfo', 'label+percent+name')
            )

        elif trace_type == 'area':
            trace = go.Scatter(
                x=trace_data.get('x', []),
                y=trace_data.get('y', []),
                mode='lines',
                name=trace_data.get('name', ''),
                fill='tozeroy',
                line=dict(color=trace_data['line']['color'], width=trace_data['line']['width']) if 'line' in trace_data else None,
                hoverinfo='text',
                text=trace_data['text'] if 'text' in trace_data else None
            )

        elif trace_type == 'scatter3d':
            trace = go.Scatter3d(
                x=trace_data.get('x', []),
                y=trace_data.get('y', []),
                z=trace_data.get('z', []),
                mode=trace_data.get('mode', 'markers'),
                name=trace_data.get('name', ''),
                marker=dict(color=trace_data['marker']['color']) if 'marker' in trace_data else None,
                line=dict(color=trace_data['line']['color'], width=trace_data['line']['width']) if 'line' in trace_data else None,
                hoverinfo='text',
                text=trace_data['text'] if 'text' in trace_data else None
            )

        elif trace_type == 'surface':
            trace = go.Surface(
                z=trace_data.get('z', []),
                name=trace_data.get('name', ''),
                colorscale=trace_data.get('colorscale', 'Viridis'),
                hoverinfo=trace_data.get('hoverinfo', 'z+name')
            )

        elif trace_type == 'box':
            trace = go.Box(
                y=trace_data.get('y', []),
                x=trace_data.get('x', []),
                name=trace_data.get('name', ''),
                marker=dict(color=trace_data['marker']['color'], size=trace_data['marker']['size']) if 'marker' in trace_data else None,
                boxmean=trace_data.get('boxmean', False),
                boxpoints=trace_data.get('boxpoints', 'all'),
                jitter=trace_data.get('jitter', 0.5),
                pointpos=trace_data.get('pointpos', -1.8),
                line=dict(color=trace_data['line']['color'], width=trace_data['line']['width']) if 'line' in trace_data else None
            )

        elif trace_type == 'histogram':
            trace = go.Histogram(
                x=trace_data.get('x', []),
                y=trace_data.get('y', []),
                name=trace_data.get('name', ''),
                marker=dict(color=trace_data['marker']['color']) if 'marker' in trace_data else None,
                hoverinfo='x+y+name'
            )

        elif trace_type == 'heatmap':
            trace = go.Heatmap(
                z=trace_data.get('z', []),
                x=trace_data.get('x', []),
                y=trace_data.get('y', []),
                name=trace_data.get('name', ''),
                colorscale=trace_data.get('colorscale', 'Viridis'),
                hoverinfo='x+y+z'
            )

        elif trace_type == 'bubble':
            trace = go.Scatter(
                x=trace_data.get('x', []),
                y=trace_data.get('y', []),
                mode='markers',
                name=trace_data.get('name', ''),
                marker=dict(
                    size=trace_data.get('marker', {}).get('size', []),
                    color=trace_data.get('marker', {}).get('color', []),
                    sizemode='area',
                    sizeref=2.*max(trace_data.get('marker', {}).get('size', []))/(40.**2),
                    sizemin=4
                ),
                hoverinfo='text',
                text=trace_data.get('text', [])
            )

        else:
            raise ValueError(f"Unsupported trace type: {trace_type}")

        traces.append(trace)

    # Create figure with the extracted layout and data
    fig = go.Figure(data=traces, layout=layout)

    # Render the figure with the config
    pio.show(fig, config=config)

    # Save the figure as an image (optional)
    fig.write_image("new_image.png")

In [8]:
# Code to get information using Gemini model

def gemini_model(prompt):
    
    genai.configure(api_key=gemini_key)

    model = genai.GenerativeModel(model_name="gemini-pro")

    template = prompt

    response = model.generate_content(template)
    return str(response.text)

In [6]:
import time
import json
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.oxml import OxmlElement
from docx.oxml.ns import qn

# Function to update or create the documentation
def update_doc_v5(doc_path, image_loc, topic, data, layout, updated_data, updated_layout, chart_path):

    # Check if the document exists or create a new one
    try:
        document = Document(doc_path)
        document.add_page_break()
    except:
        document = Document()
        document.add_heading('Chart Editing', level=1)
        document.add_page_break()
    
    # Add a new section with a title
    document.add_heading(topic, level=1)
    
    # Add the image with caption
    document.add_picture(image_loc, width=Inches(4.5))
    last_paragraph = document.paragraphs[-1]
    last_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    caption = document.add_paragraph(f'Figure: {image_loc}')
    caption.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    
    # Add Prompt
    # document.add_heading('Prompt Used', level=2)
    # prompt_paragraph = document.add_paragraph(prompt)
    # prompt_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
    
    # Add Extracted JSON
    document.add_heading('Extracted JSON', level=2)
    document.add_heading('data', level=3)
    json_paragraph = document.add_paragraph(json.dumps(data, indent=4))
    json_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
    document.add_heading('layout', level=3)
    json_paragraph = document.add_paragraph(json.dumps(layout, indent=4))
    json_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
    # document.add_heading('config', level=3)
    # json_paragraph = document.add_paragraph(json.dumps(config, indent=4))
    # json_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
    
    
    # Add Updated JSON
    document.add_heading('Updated JSON', level=2)
    document.add_heading('updated_data', level=3)
    json_paragraph = document.add_paragraph(json.dumps(updated_data, indent=4))
    json_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
    document.add_heading('updated_layout', level=3)
    json_paragraph = document.add_paragraph(json.dumps(updated_layout, indent=4))
    json_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
    # document.add_heading('updated_config', level=3)
    # json_paragraph = document.add_paragraph(json.dumps(updated_config, indent=4))
    # json_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
    
    # Add Updated Chart
    document.add_heading('Updated Chart', level=2)
    document.add_picture(chart_path, width=Inches(4.5))
    last_paragraph = document.paragraphs[-1]
    last_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    
    # # Add Performance Metrics Table
    # document.add_heading('Performance Metrics', level=2)
    # table = document.add_table(rows=1, cols=4)
    # table.style = 'Table Grid'
    
    # # Define table headers
    # headers = ["Task", "Model Used", "Description", "Time Taken (s)"]
    # hdr_cells = table.rows[0].cells
    # for i, header in enumerate(headers):
    #     hdr_cells[i].text = header
    #     hdr_cells[i].paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    #     hdr_cells[i].paragraphs[0].bold = True
    
    # # Add metrics data to the table
    # for metric in performance_metrics:
    #     row_cells = table.add_row().cells
    #     for i, value in enumerate(metric):
    #         row_cells[i].text = value
    #         row_cells[i].paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    
    # Save the document
    document.save(doc_path)
    print(f"Document saved as {doc_path}")


In [9]:
def update_info(extracted_json, prompt):
    updated_json = gemini_model(prompt+  extracted_json)
    return updated_json

In [7]:
def trail_run(image_loc,prompt_for_image,prompt_for_chart_editing,topic):
    chart_path = 'new_image.png'

    # Placeholder functions for simulation
    start_time = time.time()
    extracted_json = get_image_info(image_loc, prompt_for_image)
    time_info = time.time() - start_time
    print("Extracted Json:/n",extracted_json)
    data_json = extract_json_section(extracted_json, "data")
    layout_json = extract_json_section(extracted_json, "layout")
    config_json = extract_json_section(extracted_json, "config")
    # Convert JSON strings to Python dictionaries
    data = json.loads(data_json) if data_json else None
    layout = json.loads(layout_json) if layout_json else None
    config = json.loads(config_json) if config_json else None

    start_time = time.time()
    updated_json = update_info(extracted_json, prompt_for_chart_editing)
    time_update = time.time() - start_time
    print("Updated Json:/n",updated_json)
    updated_data_json = extract_json_section(updated_json, "data")
    updated_layout_json = extract_json_section(updated_json, "layout")
    updated_config_json = extract_json_section(updated_json,"config")
    # Convert JSON strings to Python dictionaries
    updated_data = json.loads(updated_data_json) if updated_data_json else None
    updated_layout = json.loads(updated_layout_json) if updated_layout_json else None
    updated_config = json.loads(updated_config_json) if updated_config_json else None

    plot_from_ai_output_v3(updated_json)
    # Call the update_doc function
    doc_path = 'Different_charts_trial.docx'
    update_doc_v5(doc_path, image_loc, topic, data, layout, updated_data, updated_layout, chart_path)

In [ ]:
prompt2v7 = '''   

## Task: Extract Detailed Information from a Given Graph Image. Charts are multilingual, you have to extract the same language that is there in the graph.

You will analyze a graph image and generate corresponding Plotly-compatible JSON files (`data.json`, `layout.json`, and `config.json`). Follow the structured steps and specific XML-like delimiters for each JSON section as outlined below.

### Instructions:

#### Graph Analysis:

1. **Identify and Categorize Plot Types**:
   - Recognize and categorize the plot types present in the graph image. Charts may include but are not limited to the following types:
     - **Comparison Charts**:
       - `bar`: Bar Chart
       - `bar` (with vertical bars): Column Chart
       - `bar` (grouped): Grouped Bar/Column Chart
       - `scatter` (with lines connecting to points): Lollipop Chart
       - `indicator` (with gauge): Bullet Chart
       - `scatter`: Dot Plot
       - `scatter` (with lines connecting different points): Dumbbell Chart
       - `scatter` (or custom SVG): Pictogram
       - `scatter` or `bar` with a range: Range Chart
       - `barpolar`: Radial Bar Chart
       - `parcoords`: Parallel Coordinates
       - `scatterpolar`: Radar Chart
       - `barpolar` (with segments): Nightingale Chart
       - `waterfall`: Waterfall Chart
       - `heatmap` (with color scale): Matrix Chart
       - `various charts in small multiples`: Small Multiples
       - `text` (frequency-based size): Word Cloud
       - `scatter` (with lines indicating change): Slope Chart
       - `table`: Table Chart
       - `scatter`: Categorical Scatter Plot
       - `scatter` (with four quadrants): Quadrant Chart
     - **Correlation Charts**:
       - `heatmap`: Heatmap
       - `scatter` (with size): Bubble Chart
       - `scatter`: Scatter Plot
       - `scatter` (with lines connecting points): Connected Scatter Plot
       - `hexbin`: Hexagonal Binning
       - `contour`: Contour Plot
     - **Part-to-Whole & Hierarchical Charts**:
       - `bar` (stacked): Stacked Bar/Column Chart
       - `bar` (with diverging sections): Diverging Bar Chart
       - `bar` (mirrored pairs): Population Pyramid
       - `scatter` (or custom symbols in a grid): Icon Array
       - `waffle`: Waffle Chart
       - `pie`: Pie Chart
       - `pie` (with hole): Donut Chart
       - `pie` (half-circle): Semi-circle Donut Chart
       - `bar` (variable width): Marimekko Chart
       - `treemap`: Treemap
       - `treemap` (circular): Circular Treemap
       - `treemap` (convex shape): Convex Treemap
       - `sunburst`: Sunburst Chart
       - `funnel` or `pyramid`: Funnel & Pyramid Chart
       - `dendrogram`: Dendrogram
       - `venn`: Venn Diagram
       - `euler`: Euler Diagram
       - `indicator` (circular gauge): Circular Gauge
     - **Data Over Time (Temporal) Charts**:
       - `scatter` (filled area under curve): Area Chart
       - `scatter` (filled stacked areas): Stacked Area Chart
       - `scatter` (smoothed filled areas): Stream Graph
       - `scatter` (indicating ranking change): Bump Chart
       - `scatter` (with filled area): Bump Area Chart
       - `scatter`: Line Chart
       - `scatter` (with smoothed lines): Spline Chart
       - `scatter` (with stepped lines): Step Line Chart
       - `candlestick`: Candlestick Chart
       - `gantt`: Gantt Chart
       - `scatter` (encoded lines): Barcode Chart
       - `ohlc`: OHLC Chart
     - **Distribution Charts**:
       - `densitymapbox` or `histogram`: Density Plot
       - `ridge`: Ridgeline Plot
       - `horizon`: Horizon Chart
       - `histogram`: Histogram
       - `radial_histogram`: Radial Histogram
       - `scatter` (with distribution): Strip Plot
       - `scatter` (with jitter): Jitter Plot
       - `heatmap`: One-dimensional Heatmap
       - `scatter` (with tightly packed points): Beeswarm Chart
       - `box`: Box Chart
       - `violin`: Violin Plot
     - **Geospatial & Other Charts**:
       - `densitymapbox`: Geographic Heatmap
       - `choropleth`: Choropleth Map
       - `scattermapbox`: Tile Map
       - `chord`: Chord Diagram
       - `arc`: Arc Diagram
       - `sankey`: Sankey
       - `network`: Network Diagram
       - `flowchart`: Flowchart

2. **Extract Data Points** for Each Plot Type:
   - Gather the data points for each identified plot type.
     - For `scatter`, `line`, and `area` plots: Extract `x` and `y` values.
     - For `bar` and `stacked bar` plots: Extract `x` (categories) and `y` values.
     - For `pie` charts: Extract `labels` and `values`.
     - For `scatter3d`, `surface`, and other 3D plots: Extract `x`, `y`, and `z` values.

3. **Identify Multiple Plots** on the Same Figure:
   - Note any combinations or overlays of multiple plot types on the same figure and their configurations, such as stacked plots or multiple area fills.

#### Generate JSON and CSV Outputs:

Follow the JSON formatting guidelines and wrap each section in the specified XML-like tags.

1. **Generate `data.json`**:
   - Include detailed information for each plot type:
     - `type`: Specify the plot type (e.g., `"scatter"`, `"bar"`, `"pie"`, `"area"`, `"scatter3d"`, `"surface"`).
       - For area charts, use `"type": "scatter"` and specify `"fill"`.
     - `x`, `y`, and `z` (if applicable): Arrays of data points for each axis.
     - `labels` and `values` for pie charts.
     - `mode`: Specify for line and scatter plots (e.g., `"markers"`, `"lines"`, `"markers+lines"`).
     - `name`: Legend entry for the plot.
     - `marker`: Properties such as color, size, and symbol.
     - `line`: Properties such as color, width, and dash style.
     - `text`: Hover text for each data point.
     - `hoverinfo`: Information displayed on hover (e.g., `"x+y+text"`).
     - `fill`: For area charts, specify the fill (e.g., `"tozeroy"`, `"tonexty"`).
     - `fillcolor`: The color used to fill the area.
     - `stackgroup`: For stacked bar charts, specify the stacking group.

2. **Generate `layout.json`**:
   - Define the layout properties:
     - `title`: Text and font properties for the graph title.
     - `xaxis` and `yaxis`: Configuration for axis titles, ranges, grids, and ticks.
     - `zaxis`: Configuration for 3D plots including title, range, and grid.
     - `legend`: Properties for legend orientation and positioning.
     - `margin`: Margins for left, right, bottom, top, and padding.
     - `plot_bgcolor` and `paper_bgcolor`: Background colors.
     - `scene`: Configuration for 3D scenes, including camera angles and aspect ratio.

3. **Generate `config.json`**:
   - Include settings such as:
     - `responsive`: Whether the graph is responsive.
     - `displayModeBar`: Whether the mode bar is shown.
     - `modeBarButtonsToRemove`: List of mode bar buttons to remove.
     - `scrollZoom`: Whether scrolling zoom is enabled.

4. **Generate `<csv>`**:
   - Format the CSV data as follows:
     ```csv
     x,y
     1,4
     2,5
     3,6
     ```
   - Replace `x`, `y`, `1`, `4`, `2`, `5`, and `3`, `6` with the actual data from the graph image.

### Output Format:

Wrap each JSON output in the specified XML-like tags:

- `<data> ... </data>`
- `<layout> ... </layout>`
- `<config> ... </config>`
- `<csv> ... </csv>`

Ensure the JSON format follows these rules:
- Use double quotes for all keys and string values.
- Do not include trailing commas in arrays or objects.

### Example Output:

```xml
<data>
[
  {
    "type": "scatter",
    "x": [1, 2, 3],
    "y": [4, 5, 6],
    "mode": "markers+lines",
    "name": "Line Plot",
    "marker": {
      "color": "rgba(75, 192, 192, 0.6)",
      "size": 8
    },
    "line": {
      "color": "rgba(75, 192, 192

, 1)",
      "width": 2
    }
  },
  {
    "type": "area",
    "x": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    "y": [400, 430, 550, 620, 580, 600, 630, 700, 670, 630, 600, 500],
    "name": "Housing",
    "marker": {
      "color": "rgba(255, 159, 64, 0.8)"
    },
    "line": {
      "color": "rgba(255, 159, 64, 1)",
      "width": 2
    }
  },
  {
    "type": "bar",
    "x": ["A", "B", "C"],
    "y": [10, 15, 13],
    "name": "Bar Chart",
    "marker": {
      "color": "rgba(255, 99, 132, 0.6)"
    },
    "hoverinfo": "x+y+text"
  },
  {
    "type": "scatter",
    "x": [1, 2, 3],
    "y": [6, 7, 8],
    "mode": "lines",
    "fill": "tozeroy",
    "name": "Area Chart",
    "line": {
      "color": "rgba(54, 162, 235, 1)",
      "width": 2
    },
    "fillcolor": "rgba(54, 162, 235, 0.5)"
  },
  {
    "type": "scatter3d",
    "x": [1, 2, 3],
    "y": [4, 5, 6],
    "z": [7, 8, 9],
    "mode": "markers",
    "name": "3D Scatter Plot",
    "marker": {
      "color": "rgba(255, 159, 64, 0.8)",
      "size": 5
    },
    "hoverinfo": "x+y+z+text"
  },
  {
    "type": "surface",
    "z": [
      [10, 10.625, 12.5, 15.625, 20],
      [5.625, 6.25, 8.125, 11.25, 15.625],
      [0, 1.25, 3.125, 6.25, 10.625]
    ],
    "name": "Surface Plot",
    "colorscale": "Viridis",
    "hoverinfo": "z+name"
  }
]
</data>
<layout>
{
  "title": {
    "text": "Complex Graph Example",
    "font": {
      "family": "Arial, sans-serif",
      "size": 24,
      "color": "#000000"
    }
  },
  "xaxis": {
    "title": {
      "text": "X Axis",
      "font": {
        "family": "Arial, sans-serif",
        "size": 18,
        "color": "#000000"
      }
    },
    "showgrid": true,
    "gridcolor": "rgba(0, 0, 0, 0.1)",
    "zeroline": true,
    "zerolinecolor": "rgba(0, 0, 0, 0.1)"
  },
  "yaxis": {
    "title": {
      "text": "Y Axis",
      "font": {
        "family": "Arial, sans-serif",
        "size": 18,
        "color": "#000000"
      }
    },
    "showgrid": true,
    "gridcolor": "rgba(0, 0, 0, 0.1)",
    "zeroline": true,
    "zerolinecolor": "rgba(0, 0, 0, 0.1)"
  },
  "legend": {
    "orientation": "h",
    "x": 0.5,
    "xanchor": "center",
    "y": -0.2,
    "font": {
      "family": "Arial, sans-serif",
      "size": 12,
      "color": "#000000"
    }
  },
  "margin": {
    "l": 60,
    "r": 30,
    "b": 60,
    "t": 60
  },
  "plot_bgcolor": "#ffffff",
  "paper_bgcolor": "#ffffff",
  "scene": {
    "xaxis": {"title": "X Axis"},
    "yaxis": {"title": "Y Axis"},
    "zaxis": {"title": "Z Axis"}
  }
}
</layout>
<config>
{
  "responsive": true,
  "displayModeBar": true,
  "modeBarButtonsToRemove": ["toImage"],
  "scrollZoom": true
}
</config>
<csv>
x,y
1,4
2,5
3,6
</csv>

```
'''